In [6]:
import os
import sys
import numpy as np
import pandas as pd



scores_df = pd.read_csv("/nvme1/hungdx/Lightning-hydra/logs/results/wildspoof_challenge_benchmark/ConformerTCM_MDT_LoRA_LargeCorpus_MoreElevenlabs_eval_Full_set_2.6M/spoofceleb_cnsl_lora_elevenlabs_xlsr_conformertcm_mdt_more_elevenlabs_ConformerTCM_MDT_LoRA_LargeCorpus_MoreElevenlabs_eval_Full_set_2.6M.txt", sep=" ", header=None)
scores_df.columns = ["utt_id", "spoof_score", "bonafide_score"]

protocol_df = pd.read_csv("/nvme1/hungdx/Lightning-hydra/data/spoofceleb/protocol/protocol.txt", sep=" ", header=None)
protocol_df.columns = ["utt_id", "subset", "label"]

# Merge scores_df and protocol_df
merged_df = pd.merge(scores_df, protocol_df, on="utt_id", how="left")

# Filter only subset = "eval"
eval_merged_df = merged_df[merged_df["subset"] == "eval"]
train_merged_df = merged_df[merged_df["subset"] == "train"]
dev_merged_df = merged_df[merged_df["subset"] == "dev"]


# Calculate EER
from eval_metrics_DF import compute_eer

print("Train SET")
bona_cm = train_merged_df[train_merged_df['label'] == 'bonafide']['bonafide_score'].values
spoof_cm = train_merged_df[train_merged_df['label'] == 'spoof']['bonafide_score'].values

eer, threshold = compute_eer(bona_cm, spoof_cm)
print(f"EER: {eer * 100:.2f}%")

print("Dev SET")
bona_cm = dev_merged_df[dev_merged_df['label'] == 'bonafide']['bonafide_score'].values
spoof_cm = dev_merged_df[dev_merged_df['label'] == 'spoof']['bonafide_score'].values

eer, threshold = compute_eer(bona_cm, spoof_cm)
print(f"EER: {eer * 100:.2f}%")

print("EVAL SET")
bona_cm = eval_merged_df[eval_merged_df['label'] == 'bonafide']['bonafide_score'].values
spoof_cm = eval_merged_df[eval_merged_df['label'] == 'spoof']['bonafide_score'].values

eer, threshold = compute_eer(bona_cm, spoof_cm)
print(f"EER: {eer * 100:.2f}%")


Train SET
EER: 4.48%
Dev SET
EER: 4.11%
EVAL SET
EER: 4.28%
